In [1]:
import pandas as pd
import json
from datetime import datetime

# Load JSON files
with open("/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100/ABNB_10Y_DAILY.json", "r") as f:
    daily_data = json.load(f)

with open("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json", "r") as f:
    quarterly_data = json.load(f)

# Convert daily data to DataFrame
daily_df = pd.DataFrame(daily_data)
daily_df['date'] = pd.to_datetime(daily_df['date'])

# Convert quarterly data to DataFrame
quarterly_df = pd.DataFrame(quarterly_data)
quarterly_df['date'] = pd.to_datetime(quarterly_df['date'])

# Define helper to assign quarter based on date
def get_quarter(d):
    month = d.month
    if month in [1, 2, 3]:
        return "Q1"
    elif month in [4, 5, 6]:
        return "Q2"
    elif month in [7, 8, 9]:
        return "Q3"
    else:
        return "Q4"

# Add year and quarter columns to daily_df
daily_df['year'] = daily_df['date'].dt.year
daily_df['quarter'] = daily_df['date'].apply(get_quarter)

# Add year and quarter columns to quarterly_df for merging
quarterly_df['year'] = quarterly_df['calendarYear'].astype(int)
quarterly_df['quarter'] = quarterly_df['period']

# Merge on year and quarter
merged_df = pd.merge(daily_df, quarterly_df, how='left', on=['year', 'quarter'])

# Drop unnecessary columns (optional)
merged_df.drop(columns=['calendarYear', 'period'], inplace=True)

# View result
merged_df


,date_x,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,2020-12-10 00:00:00+00:00,144.71,165.0000,141.2500,146.00,70447537,144.71,165.0000,141.2500,146.00,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
1,2020-12-11 00:00:00+00:00,139.25,151.5000,135.1000,146.55,26980830,139.25,151.5000,135.1000,146.55,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
2,2020-12-14 00:00:00+00:00,130.00,135.3000,125.1601,135.00,16966089,130.00,135.3000,125.1601,135.00,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
3,2020-12-15 00:00:00+00:00,124.80,127.6000,121.5000,126.69,10914432,124.80,127.6000,121.5000,126.69,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
4,2020-12-16 00:00:00+00:00,137.99,142.0000,124.9100,125.83,20409551,137.99,142.0000,124.9100,125.83,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,2025-01-27 00:00:00+00:00,128.96,129.0600,125.6400,125.94,4806967,128.96,129.0600,125.6400,125.94,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1036,2025-01-28 00:00:00+00:00,131.05,131.4200,126.8600,128.67,5929900,131.05,131.4200,126.8600,128.67,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1037,2025-01-29 00:00:00+00:00,132.58,134.0500,130.8900,131.42,3627926,132.58,134.0500,130.8900,131.42,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1038,2025-01-30 00:00:00+00:00,131.79,134.3714,130.8500,133.56,2440541,131.79,134.3714,130.8500,133.56,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961


In [8]:
import json

def load_fundamentals_as_timesteps(fundamentals_json_path, use='date'):
    with open(fundamentals_json_path, 'r') as f:
        fundamentals = json.load(f)
    
    results = []
    for entry in fundamentals:
        timestamp = entry.get(use)
        # Extract only the ratio fields (exclude metadata)
        ratio_fields = {
            k: v for k, v in entry.items()
            if k not in {"symbol", "date", "calendarYear", "period"}
        }

        ratio_vector = list(ratio_fields.values())
        results.append([timestamp, ratio_vector])

    return results


samples = load_fundamentals_as_timesteps("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json")
print(f"Total samples: {len(samples[0][1])}")
print("Example sample:", samples[0])

Total samples: 54
Example sample: ['2025-03-29', [0.8208700223419635, 0.7775072455748386, 0.19479702014926922, 46.999444205581014, 11.174245425017824, 58.17368963059884, 96.47746177612295, -38.303772145524114, 0.47050619238876246, 0.31029058610094484, 0.3073648003859101, 0.25986010759341016, 0.15455475946775846, 0.07481138654663032, 0.3709802982214504, 0.1585164629115728, 0.8454452405322416, 0.9905708202372503, 0.31029058610094484, 0.2964257788324231, 1.4699383196598599, 0.5404851336662951, 0.5951315901128608, 0, 0.24394516529851507, 4.958874782921133, 1.9149162616972568, 0.9328603628570373, 8.054235125219334, 2.034281935318713, 0.2878910011985521, 1.5974302394771485, 1.392616100138708, 3.234476108640729, 0.1516545601291364, 0.25117713063266184, 0.8717852371409486, 0.24394516529851507, 1.2207951070336391, 7.799413871703028, 3.5073949333723826, 0.1516545601291364, 48.913265282352235, 48.913265282352235, 34.262214031187405, 32.962171789749796, 156.4681034337436, 136.40658265698062, 136.4

In [9]:
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from random import randint

def load_json(path):
    """Load a JSON file."""
    with open(path) as f:
        return json.load(f)

def parse_date(date_str):
    """Convert 'YYYY-MM-DD' string to datetime object."""
    return datetime.strptime(date_str, "%Y-%m-%d")

def find_future_pct_change(start_date_str, price_dict, days_forward=30):
    """
    Given a date string and price dictionary, return percentage change
    in price after `days_forward` days.
    """
    start_date = parse_date(start_date_str)
    target_date = start_date + timedelta(days=days_forward)

    # Convert to pandas series for easy date handling
    price_series = pd.Series(price_dict).sort_index()
    price_series.index = pd.to_datetime(price_series.index)

    try:
        p_start = price_series.loc[start_date:].iloc[0]
        p_end = price_series.loc[target_date:].iloc[0]
        return (p_end - p_start) / p_start
    except IndexError:
        return None  # Not enough future data

def build_dataset(fundamentals_path, prices_path, min_k=4, max_k=24, threshold=0.05):
    """
    Load fundamentals and prices and create a labeled dataset:
    - Each sample = [ratios_t0, ratios_t1, ..., ratios_tk]
    - Label = 1 if price goes up > threshold in 30 days after t_k
    """

    # Step 1: Load and sort fundamentals
    fundamentals = load_json(fundamentals_path)
    fundamentals.sort(key=lambda x: parse_date(x["date"]))

    # Step 2: Load price data
    prices = load_json(prices_path)

    dataset = []

    for i in range(min_k, len(fundamentals)):
        k = randint(min_k, min(max_k, i))  # Random history window length

        sample_window = fundamentals[i - k:i]  # Past k quarters
        tk_entry = sample_window[-1]          # Most recent quarter
        tk_date = tk_entry["date"]

        # Step 3: Look 30 days into the future from t_k
        pct_change = find_future_pct_change(tk_date, prices)

        if pct_change is None:
            continue  # Skip if future price is missing

        label = 1 if pct_change > threshold else 0

        # Step 4: Extract only the ratios, discard date
        sequence = [entry["ratios"] for entry in sample_window]

        dataset.append((sequence, label))

    return dataset


In [10]:
dataset = build_dataset("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json", "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100/ABNB_10Y_DAILY.json", min_k=8, max_k=18)

# Separate into input and output
X = [x[0] for x in dataset]
y = [x[1] for x in dataset]

print(f"Total samples: {len(X)}")
print(f"Input shape: {len(X[0])} timesteps × {len(X[0][0])} features")
print(f"Example label: {y[0]}")


Total samples: 0


IndexError: list index out of range

In [11]:
import json
import pandas as pd
from datetime import datetime, timedelta
from random import randint

# ---------------------
# Load and Prepare Data
# ---------------------

def load_data(fundamentals_path, prices_path):
    with open(fundamentals_path) as f:
        fundamentals_raw = json.load(f)
    with open(prices_path) as f:
        prices_raw = json.load(f)

    # Process fundamentals: extract ratios and date
    fundamentals = []
    for entry in fundamentals_raw:
        date = entry["date"][:10]  # YYYY-MM-DD
        ratios = [v for k, v in entry.items() if k not in {"symbol", "date", "calendarYear", "period"}]
        fundamentals.append({"date": date, "ratios": ratios})

    # Process prices: map date -> adjusted close price
    prices = {
        datetime.strptime(entry["date"][:10], "%Y-%m-%d").strftime("%Y-%m-%d"): entry["adjClose"]
        for entry in prices_raw
    }

    return fundamentals, prices

# --------------------------
# Utility for Price Labeling
# --------------------------

def find_future_price_change(start_date_str, price_dict, days_forward=30):
    """Return percentage price change after `days_forward` days from `start_date_str`."""
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    target_date = start_date + timedelta(days=days_forward)

    # Convert price dict to time series
    price_series = pd.Series(price_dict).sort_index()
    price_series.index = pd.to_datetime(price_series.index)

    try:
        price_start = price_series.loc[start_date:].iloc[0]
        price_end = price_series.loc[target_date:].iloc[0]
        return (price_end - price_start) / price_start
    except IndexError:
        return None  # Not enough future data

# --------------------------
# Dataset Builder
# --------------------------

def build_labeled_dataset(fundamentals, prices, min_k=4, max_k=24, price_threshold=0.05):
    """
    Returns a list of tuples: (sequence_of_ratios, label)
    - Each input sequence is a list of k ratio vectors.
    - Label is 1 if price increased by > threshold after last quarter, else 0.
    """
    fundamentals.sort(key=lambda x: datetime.strptime(x["date"], "%Y-%m-%d"))
    dataset = []

    for i in range(min_k, len(fundamentals)):
        k = randint(min_k, min(max_k, i))
        window = fundamentals[i - k:i]  # k past quarters
        end_date = window[-1]["date"]

        # Calculate price movement after end_date
        price_change = find_future_price_change(end_date, prices)
        if price_change is None:
            continue  # Skip if price data not available

        label = 1 if price_change > price_threshold else 0
        sequence = [entry["ratios"] for entry in window]
        dataset.append((sequence, label))

    return dataset


In [13]:
fundamentals, prices = load_data("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json", "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100/ABNB_10Y_DAILY.json")
dataset = build_labeled_dataset(fundamentals, prices, min_k=4, max_k=18)

# Inspect results
print(f"Total samples: {len(dataset)}")
print("Example sample:")
print("Input (sequence of ratios):", dataset[0][0])
print("Label (BUY=1, SELL=0):", dataset[0][1])


Total samples: 155
Example sample:
Input (sequence of ratios): [[2.7830060934326335, 2.217670954637779, 1.140825998645904, 48.37197949719307, 0, 48.37197949719307, 0, 48.37197949719307, 1, -3.3226751281425435, -6.645350256285087, 0, 1, 0, 0, -2.1243757802746566, 0, 2, -3.3226751281425435, 0, 0, 0, 0, 0, 0, 1.7006357856494096, 1.8605812897366032, 0, 0, 4.532079646017699, 0.43762016663106174, None, None, 0.024324402355930246, 0, None, 0, 0, 0, 0, 0, 0, 1.7695378252497729, 1.7695378252497729, 2.3776679834024894, 0, 0, 0, 0, 0, 2.3776679834024894, None, -0.46803097979872177, 1.7695378252497729], [0, 0, 0, None, 0, None, 0, None, 1, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, None, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 2.5455479421914213, 0, 0, 0, 0, 0, 2.5455479421914213, None, None, 0], [0, 0, 0, None, 0, None, 0, None, 1, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, None, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 4.328485282465151, 0, 

In [3]:
import os
import json
import pandas as pd
from pathlib import Path

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                df['__source_file__'] = file.name
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def perform_eda(df):
    eda = {
        'shape': df.shape,
        'columns': df.columns.tolist(),
        'data_types': df.dtypes.to_dict(),
        'missing_values': df.isnull().sum().to_dict(),
        'summary_stats': df.describe(include='all').to_dict(),
        'unique_counts': df.nunique().to_dict()
    }
    return eda


def summarize_folder(folder_name, dfs):
    print(f"\n=== Summary for Folder: {folder_name} ===\n")
    total_files = len(dfs)
    print(f"Total JSON files read: {total_files}")

    combined_df = pd.concat(dfs, ignore_index=True)
    eda_result = perform_eda(combined_df)

    print("\n>> Shape of combined data:", eda_result['shape'])
    print("\n>> Data Types:\n", pd.Series(eda_result['data_types']))
    print("\n>> Missing Values:\n", pd.Series(eda_result['missing_values']))
    print("\n>> Unique Value Counts:\n", pd.Series(eda_result['unique_counts']))
    print("\n>> Summary Statistics:\n", pd.DataFrame(eda_result['summary_stats']))

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue
        dfs = load_json_files_from_folder(folder)
        if dfs:
            summarize_folder(folder, dfs)
        else:
            print(f"No readable JSON files in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)



=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Total JSON files read: 101

>> Shape of combined data: (10893, 59)

>> Data Types:
 symbol                                 object
date                                   object
calendarYear                           object
period                                 object
currentRatio                          float64
quickRatio                            float64
cashRatio                             float64
daysOfSalesOutstanding                float64
daysOfInventoryOutstanding            float64
operatingCycle                        float64
daysOfPayablesOutstanding             float64
cashConversionCycle                   float64
grossProfitMargin                     float64
operatingProfitMargin                 float64
pretaxProfitMargin                    float64
netProfitMargin                       float64
effectiveTaxRate                      float64
returnOnAssets     

In [4]:
import os
import json
import pandas as pd
from pathlib import Path
from tabulate import tabulate

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                df['__source_file__'] = file.name
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def perform_eda(df):
    return {
        'Shape': df.shape,
        'Columns': df.columns.tolist(),
        'Data Types': df.dtypes.to_dict(),
        'Missing Values': df.isnull().sum().to_dict(),
        'Unique Counts': df.nunique().to_dict(),
        'Summary Stats': df.describe(include='all').to_dict()
    }

def tabulate_eda(folder_name, eda):
    print(f"\n=== EDA Summary for Folder: {folder_name} ===\n")

    print(f"Shape: {eda['Shape']}")
    print("\n>> Data Types:")
    print(tabulate(eda['Data Types'].items(), headers=['Column', 'Type'], tablefmt='grid'))

    print("\n>> Missing Values:")
    print(tabulate(eda['Missing Values'].items(), headers=['Column', 'Missing Count'], tablefmt='grid'))

    print("\n>> Unique Value Counts:")
    print(tabulate(eda['Unique Counts'].items(), headers=['Column', 'Unique Values'], tablefmt='grid'))

    print("\n>> Summary Statistics:")
    stats_df = pd.DataFrame(eda['Summary Stats'])
    print(tabulate(stats_df, headers='keys', tablefmt='grid', showindex=True))

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue
        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            eda = perform_eda(combined_df)
            tabulate_eda(folder, eda)
        else:
            print(f"No readable JSON files in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)



=== EDA Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Shape: (10893, 59)

>> Data Types:
+------------------------------------+---------+
| Column                             | Type    |
+====================================+=========+
| symbol                             | object  |
+------------------------------------+---------+
| date                               | object  |
+------------------------------------+---------+
| calendarYear                       | object  |
+------------------------------------+---------+
| period                             | object  |
+------------------------------------+---------+
| currentRatio                       | float64 |
+------------------------------------+---------+
| quickRatio                         | float64 |
+------------------------------------+---------+
| cashRatio                          | float64 |
+------------------------------------+---------+
| daysOfSales

In [5]:
import os
import json
import pandas as pd
from pathlib import Path
from tabulate import tabulate

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def clean_and_analyze(df):
    original_missing = df.isnull().sum()
    missing_features = (original_missing > 0).sum()

    # Simple substitution: fill numeric with mean, categorical with mode
    substituted = 0
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
                substituted += 1
            elif df[col].dtype == 'object':
                df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "unknown", inplace=True)
                substituted += 1

    # Detect label column
    label_col = None
    for col in df.columns:
        if set(df[col].unique()).intersection({'buy', 'sell', 'hold'}):
            label_col = col
            break

    if label_col:
        class_counts = df[label_col].value_counts(normalize=True) * 100
    else:
        class_counts = pd.Series(dtype=float)

    return df.shape, missing_features, substituted, label_col, class_counts.round(2)

def summarize_folder(folder_path, df):
    shape, missing_count, substituted_count, label_col, class_dist = clean_and_analyze(df)

    print(f"\n=== Summary for Folder: {folder_path} ===")
    print(f"Dataset Shape: {shape}")
    print(f"Features with Missing Values: {missing_count}")
    print(f"Features Substituted (Filled): {substituted_count}")
    
    if label_col:
        print(f"\nClass Balance from column `{label_col}`:")
        print(tabulate(class_dist.reset_index().values, headers=["Class", "Percentage (%)"], tablefmt='grid'))
    else:
        print("No label column with buy/sell/hold found.")

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue

        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            summarize_folder(folder, combined_df)
        else:
            print(f"No JSON files found in {folder}")

# === Usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)


/tmp/ipykernel_284462/3225011517.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)



=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===
Dataset Shape: (10893, 58)
Features with Missing Values: 23
Features Substituted (Filled): 23
No label column with buy/sell/hold found.

=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100 ===
Dataset Shape: (613437, 13)
Features with Missing Values: 0
Features Substituted (Filled): 0
No label column with buy/sell/hold found.


In [6]:
import os
import json
import pandas as pd
from pathlib import Path
from tabulate import tabulate

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def generate_label(df, price_col='close', date_col='date'):
    if date_col not in df.columns or price_col not in df.columns:
        print("Missing required columns for labeling.")
        return df

    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(by=date_col, inplace=True)

    # Calculate percent change over 65 days (quarterly)
    df['quarterly_change'] = df[price_col].pct_change(periods=65) * 100

    def classify(change):
        if pd.isna(change):
            return None
        elif change > 5:
            return 'buy'
        elif change < -5:
            return 'sell'
        else:
            return 'hold'

    df['label'] = df['quarterly_change'].apply(classify)
    return df

def analyze(df):
    original_missing = df.isnull().sum()
    missing_features = (original_missing > 0).sum()

    substituted = 0
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
                substituted += 1
            elif df[col].dtype == 'object':
                df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "unknown", inplace=True)
                substituted += 1

    class_counts = df['label'].value_counts(normalize=True) * 100 if 'label' in df.columns else pd.Series(dtype=float)

    return df.shape, missing_features, substituted, class_counts.round(2)

def summarize_folder(folder_path, df):
    df = generate_label(df)

    shape, missing_count, substituted_count, class_dist = analyze(df)

    print(f"\n=== Summary for Folder: {folder_path} ===")
    print(f"Dataset Shape: {shape}")
    print(f"Features with Missing Values: {missing_count}")
    print(f"Features Substituted (Filled): {substituted_count}")
    
    if not class_dist.empty:
        print("\nClass Balance based on derived label (quarterly price change):")
        print(tabulate(class_dist.reset_index().values, headers=["Label", "Percentage (%)"], tablefmt='grid'))
    else:
        print("No valid label could be generated.")

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue

        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            summarize_folder(folder, combined_df)
        else:
            print(f"No JSON files found in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)


Missing required columns for labeling.

=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===
Dataset Shape: (10893, 58)
Features with Missing Values: 23
Features Substituted (Filled): 23
No valid label could be generated.


/tmp/ipykernel_284462/3689802100.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)



=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100 ===
Dataset Shape: (613437, 15)
Features with Missing Values: 2
Features Substituted (Filled): 2

Class Balance based on derived label (quarterly price change):
+---------+------------------+
| Label   |   Percentage (%) |
+=========+==================+
| buy     |            48.29 |
+---------+------------------+
| sell    |            48.21 |
+---------+------------------+
| hold    |             3.51 |
+---------+------------------+


/tmp/ipykernel_284462/3689802100.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/tmp/ipykernel_284462/3689802100.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [7]:
import os
import json
import pandas as pd
from pathlib import Path

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                df['__source_file__'] = file.name
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def perform_eda(df):
    eda = {
        'Shape': df.shape,
        'Columns': df.columns.tolist(),
        'Data Types': pd.Series(df.dtypes).reset_index().rename(columns={'index': 'Column', 0: 'Type'}),
        'Missing Values': df.isnull().sum().reset_index().rename(columns={'index': 'Column', 0: 'Missing Count'}),
        'Unique Counts': df.nunique().reset_index().rename(columns={'index': 'Column', 0: 'Unique Values'}),
        'Summary Stats': df.describe(include='all').transpose()
    }
    return eda

def display_eda(folder_name, eda):
    print(f"\n=== EDA Summary for Folder: {folder_name} ===\n")

    print(f"Shape: {eda['Shape']}")
    print("\nColumns:")
    print(eda['Columns'])

    print("\n>> Data Types:")
    display(eda['Data Types'])

    print("\n>> Missing Values:")
    display(eda['Missing Values'])

    print("\n>> Unique Value Counts:")
    display(eda['Unique Counts'])

    print("\n>> Summary Statistics:")
    display(eda['Summary Stats'])

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue
        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            eda = perform_eda(combined_df)
            display_eda(folder, eda)
        else:
            print(f"No readable JSON files in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)



=== EDA Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Shape: (10893, 59)

Columns:
['symbol', 'date', 'calendarYear', 'period', 'currentRatio', 'quickRatio', 'cashRatio', 'daysOfSalesOutstanding', 'daysOfInventoryOutstanding', 'operatingCycle', 'daysOfPayablesOutstanding', 'cashConversionCycle', 'grossProfitMargin', 'operatingProfitMargin', 'pretaxProfitMargin', 'netProfitMargin', 'effectiveTaxRate', 'returnOnAssets', 'returnOnEquity', 'returnOnCapitalEmployed', 'netIncomePerEBT', 'ebtPerEbit', 'ebitPerRevenue', 'debtRatio', 'debtEquityRatio', 'longTermDebtToCapitalization', 'totalDebtToCapitalization', 'interestCoverage', 'cashFlowToDebtRatio', 'companyEquityMultiplier', 'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover', 'fixedAssetTurnover', 'assetTurnover', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare', 'payoutRatio', 'operatingCashFlowSalesRatio', 'freeCashFlowOperatingCashFlowRatio'

,Column,Type
0,symbol,object
1,date,object
2,calendarYear,object
3,period,object
4,currentRatio,float64
5,quickRatio,float64
6,cashRatio,float64
7,daysOfSalesOutstanding,float64
8,daysOfInventoryOutstanding,float64
9,operatingCycle,float64



>> Missing Values:


,Column,Missing Count
0,symbol,0
1,date,0
2,calendarYear,0
3,period,0
4,currentRatio,0
5,quickRatio,0
6,cashRatio,0
7,daysOfSalesOutstanding,615
8,daysOfInventoryOutstanding,78
9,operatingCycle,615



>> Unique Value Counts:


...

,Column,Missing Count
0,date,0
1,close,0
2,high,0
3,low,0
4,open,0
5,volume,0
6,adjClose,0
7,adjHigh,0
8,adjLow,0
9,adjOpen,0



>> Unique Value Counts:


,Column,Unique Values
0,date,13889
1,close,179089
2,high,193002
3,low,191932
4,open,175627
5,volume,359783
6,adjClose,395954
7,adjHigh,397220
8,adjLow,396360
9,adjOpen,389231



>> Summary Statistics:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,613437,13889,2025-01-17T00:00:00.000Z,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
close,613437.0,NaN,NaN,NaN,1350.928252,5347.011428,0.62,51.38,161.44,739.17,134657.28
high,613437.0,NaN,NaN,NaN,1366.971984,5402.132968,0.62,52.25,164.0,749.0,134884.8
low,613437.0,NaN,NaN,NaN,1334.052841,5287.437571,0.5938,50.5,158.999921,729.0,133764.48
open,613437.0,NaN,NaN,NaN,1350.801673,5346.574885,0.62,51.375,161.48,739.06,134496.0
volume,613437.0,NaN,NaN,NaN,5297937.849491,20761003.624807,0.0,478745.0,1596735.0,4317700.0,9232320000.0
adjClose,613437.0,NaN,NaN,NaN,68.141743,187.949652,0.006458,5.427208,18.778898,57.453541,5289.474426
adjHigh,613437.0,NaN,NaN,NaN,69.027742,190.152543,0.006458,5.523434,19.065,58.176297,5326.298782
adjLow,613437.0,NaN,NaN,NaN,67.221941,185.678783,0.006185,5.328526,18.477476,56.67,5249.217122
adjOpen,613437.0,NaN,NaN,NaN,68.135584,187.934149,0.006458,5.427208,18.779884,57.410424,5289.135123


In [8]:
import os
import json
import pandas as pd
from pathlib import Path

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def generate_label(df, price_col='close', date_col='date'):
    if date_col not in df.columns or price_col not in df.columns:
        print("Missing required columns for labeling.")
        return df

    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(by=date_col, inplace=True)

    # Calculate percent change over 65 days (quarterly)
    df['quarterly_change'] = df[price_col].pct_change(periods=65) * 100

    def classify(change):
        if pd.isna(change):
            return None
        elif change > 5:
            return 'buy'
        elif change < -5:
            return 'sell'
        else:
            return 'hold'

    df['label'] = df['quarterly_change'].apply(classify)
    return df

def analyze(df):
    original_missing = df.isnull().sum()
    missing_features = (original_missing > 0).sum()

    substituted = 0
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
                substituted += 1
            elif df[col].dtype == 'object':
                df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "unknown", inplace=True)
                substituted += 1

    class_counts = df['label'].value_counts(normalize=True) * 100 if 'label' in df.columns else pd.Series(dtype=float)

    return df.shape, missing_features, substituted, class_counts.round(2)

def summarize_folder(folder_path, df):
    df = generate_label(df)

    shape, missing_count, substituted_count, class_dist = analyze(df)

    print(f"\n=== Summary for Folder: {folder_path} ===")
    
    # Display as pandas objects
    shape_df = pd.DataFrame({'Dataset Shape': [shape]})
    missing_df = pd.DataFrame({'Features with Missing Values': [missing_count]})
    substituted_df = pd.DataFrame({'Features Substituted (Filled)': [substituted_count]})
    
    print("\nDataset Shape:")
    display(shape_df)

    print("\nFeatures with Missing Values:")
    display(missing_df)

    print("\nFeatures Substituted (Filled):")
    display(substituted_df)

    if not class_dist.empty:
        print("\nClass Balance based on derived label (quarterly price change):")
        display(class_dist.reset_index().rename(columns={'index': 'Label', 'label': 'Percentage (%)'}))
    else:
        print("No valid label could be generated.")

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue

        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            summarize_folder(folder, combined_df)
        else:
            print(f"No JSON files found in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)


Missing required columns for labeling.

=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Dataset Shape:


/tmp/ipykernel_284462/810099777.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


,Dataset Shape
0,"(10893, 58)"



Features with Missing Values:


,Features with Missing Values
0,23


...

,Features with Missing Values
0,2



Features Substituted (Filled):


,Features Substituted (Filled)
0,2



Class Balance based on derived label (quarterly price change):


,Percentage (%),proportion
0,buy,48.29
1,sell,48.21
2,hold,3.51


In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from docx import Document
from docx.shared import Inches

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def generate_label(df, price_col='close', date_col='date'):
    if date_col not in df.columns or price_col not in df.columns:
        print("Missing required columns for labeling.")
        return df

    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(by=date_col, inplace=True)

    # Calculate percent change over 65 days (quarterly)
    df['quarterly_change'] = df[price_col].pct_change(periods=65) * 100

    def classify(change):
        if pd.isna(change):
            return None
        elif change > 5:
            return 'buy'
        elif change < -5:
            return 'sell'
        else:
            return 'hold'

    df['label'] = df['quarterly_change'].apply(classify)
    return df

def analyze(df):
    original_missing = df.isnull().sum()
    missing_features = (original_missing > 0).sum()

    substituted = 0
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
                substituted += 1
            elif df[col].dtype == 'object':
                df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "unknown", inplace=True)
                substituted += 1

    class_counts = df['label'].value_counts(normalize=True) * 100 if 'label' in df.columns else pd.Series(dtype=float)

    return df.shape, missing_features, substituted, class_counts.round(2)

def add_table_to_doc(doc, df, title):
    doc.add_heading(title, level=3)
    table = doc.add_table(rows=1, cols=len(df.columns))
    table.style = 'Table Grid'  # safer built-in style
    
    hdr_cells = table.rows[0].cells
    for i, col_name in enumerate(df.columns):
        hdr_cells[i].text = str(col_name)
    
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)
    doc.add_paragraph()


def summarize_folder_to_doc(doc, folder_path, df):
    df = generate_label(df)

    shape, missing_count, substituted_count, class_dist = analyze(df)

    doc.add_heading(f"Summary for Folder: {folder_path}", level=2)

    # Dataset Shape
    shape_df = pd.DataFrame({'Dataset Shape': [f"{shape[0]} rows x {shape[1]} columns"]})
    add_table_to_doc(doc, shape_df, "Dataset Shape")

    # Missing features count
    missing_df = pd.DataFrame({'Features with Missing Values': [missing_count]})
    add_table_to_doc(doc, missing_df, "Missing Features Count")

    # Substituted features count
    substituted_df = pd.DataFrame({'Features Substituted (Filled)': [substituted_count]})
    add_table_to_doc(doc, substituted_df, "Substituted Features Count")

    # Class distribution
    if not class_dist.empty:
        class_df = class_dist.reset_index().rename(columns={'index': 'Label', 'label': 'Percentage (%)'})
        add_table_to_doc(doc, class_df, "Class Balance (Label Distribution)")
    else:
        doc.add_paragraph("No valid label could be generated.")
    doc.add_page_break()

def run_eda_on_folders_and_save_doc(folder1, folder2, output_docx_path):
    doc = Document()
    doc.add_heading('EDA Summary Report', level=1)

    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            doc.add_paragraph(f"Folder not found: {folder}")
            continue

        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            summarize_folder_to_doc(doc, folder, combined_df)
        else:
            print(f"No JSON files found in {folder}")
            doc.add_paragraph(f"No JSON files found in {folder}")

    doc.save(output_docx_path)
    print(f"Report saved to {output_docx_path}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    output_doc_path = "/home/baskar/Desktop/eda_summary_report.docx"
    run_eda_on_folders_and_save_doc(folder1_path, folder2_path, output_doc_path)


Missing required columns for labeling.


/tmp/ipykernel_284462/3571241372.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


KeyError: "no style with name 'LightListAccent1'"